## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_key

# Configure gmaps API key
gmaps.configure(api_key=google_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tautira,PF,-17.7333,-149.1500,78.82,79,100,16.37,overcast clouds
1,1,Umba,RU,66.6871,34.3429,42.64,81,56,7.43,broken clouds
2,2,Quatre Cocos,MU,-20.2078,57.7625,77.09,77,76,11.63,broken clouds
3,3,Mataura,NZ,-46.1927,168.8643,42.30,85,100,1.99,overcast clouds
4,4,Remedios,CU,22.4947,-79.5458,70.72,89,2,2.19,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df 

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tautira,PF,-17.7333,-149.1500,78.82,79,100,16.37,overcast clouds
2,2,Quatre Cocos,MU,-20.2078,57.7625,77.09,77,76,11.63,broken clouds
5,5,Sibiti,CG,-3.6819,13.3499,77.81,76,83,3.36,broken clouds
6,6,Kieta,PG,-6.2167,155.6333,81.16,75,99,1.05,overcast clouds
11,11,Itarema,BR,-2.9248,-39.9167,78.57,72,15,14.18,few clouds
...,...,...,...,...,...,...,...,...,...,...
685,685,Brownsville,US,25.9018,-97.4975,82.35,86,40,4.00,scattered clouds
689,689,Labuhan,ID,-6.8844,112.2051,85.96,61,76,12.57,broken clouds
690,690,Boa Vista,BR,2.8197,-60.6733,78.78,89,0,2.30,clear sky
694,694,San Policarpo,PH,12.1791,125.5072,81.90,80,97,6.40,light rain


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                202
City                   202
Country                200
Lat                    202
Lng                    202
Max Temp               202
Humidity               202
Cloudiness             202
Wind Speed             202
Current Description    202
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

preferred_cities_df.count()

City_ID                200
City                   200
Country                200
Lat                    200
Lng                    200
Max Temp               200
Humidity               200
Cloudiness             200
Wind Speed             200
Current Description    200
dtype: int64

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tautira,PF,78.82,overcast clouds,-17.7333,-149.1500,
2,Quatre Cocos,MU,77.09,broken clouds,-20.2078,57.7625,
5,Sibiti,CG,77.81,broken clouds,-3.6819,13.3499,
6,Kieta,PG,81.16,overcast clouds,-6.2167,155.6333,
11,Itarema,BR,78.57,few clouds,-2.9248,-39.9167,
12,Cilegon,ID,89.49,broken clouds,-6.0174,106.0538,
18,Pozo Colorado,PY,76.93,overcast clouds,-23.4923,-58.7972,
23,Cape Town,ZA,76.12,overcast clouds,-33.9258,18.4232,
34,Bengkulu,ID,78.71,heavy intensity rain,-3.8004,102.2655,
37,Port Elizabeth,ZA,76.10,broken clouds,-33.9180,25.5701,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
  # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
         
print("done")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
done


In [42]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

hotel_df.count()
#hotel count dropped from 200 to 186 

City                   186
Country                186
Max Temp               186
Current Description    186
Lat                    186
Lng                    186
Hotel Name             186
dtype: int64

In [57]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))